# Sagemaker Card Classification

This notebook attempts to do my own job of image classification. This notebook pulls a list of  photos of playing cards and creates a classification model that can be used to predict the card that is in the photo.

This notebook is based off the following sources:

[1] https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/imageclassification_mscoco_multi_label/Image-classification-multilabel-lst.ipynb

[2] https://github.com/aws-samples/aws-deeplens-reinvent-2019-workshops/blob/master/AIM405-Advanced/Lab2/lab2-image-classification.ipynb

Begin with setting up some standard stuff to run Sagemaker

In [ ]:
%%time
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = 'card'

print('using bucket %s'%bucket)

Get the image classification training image used for training

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")
print (training_image)

# Data preparation

We begin by cloning a github repo containing playing cards. These are the cards we'll be using for our trainng.

Extract the data and save to the notebook.

In [ ]:
!pip install tqdm
!pip install pycocotools
!pip install scikit-image
!pip install scikit-learn

In [ ]:
%%bash
git clone https://github.com/lordloh/playing-cards.git

Now create a data file that contains the image and the category it belongs to. These files will be used as the input data to our model

In [ ]:
import random
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util

def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 90% on the left and 90% on the right
    random_degree = random.uniform(-90, 90)
    return sk.transform.rotate(image_array, random_degree)

def random_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array)

We don't have enough images, so generate some new images using data augmentation per this post: 

https://medium.com/@thimblot/data-augmentation-boost-your-image-dataset-with-few-lines-of-python-155c2dc1baec

In [ ]:
import os
import glob
from pycocotools.coco import COCO
import random
import skimage.io

DATA_DIR = './playing-cards/img'
num_files_desired = 1000

SEARCH_CRITERION = '**/*.jpg'
base_images = glob.glob(os.path.join(DATA_DIR, SEARCH_CRITERION), recursive=True)

available_transformations = {
    'rotate': random_rotation,
    'noise': random_noise
}

num_generated_files = 0
while num_generated_files <= num_files_desired:
    image_path = random.choice(base_images)
    image_to_transform = sk.io.imread(image_path)
    
    num_transformations_to_apply = random.randint(1, len(available_transformations))

    num_transformations = 0
    
    while num_transformations <= num_transformations_to_apply:
        # choose a random transformation to apply for a single image
        key = random.choice(list(available_transformations))
        transformed_image = available_transformations[key](image_to_transform)
        num_transformations += 1
        
    # define a name for our new file
    new_file_path = '%s/%s-aug-%s.jpg' % (DATA_DIR, image_path[20:34], random.randrange(1,50000))

    # write image to the disk
    sk.io.imsave(new_file_path, transformed_image)
    num_generated_files += 1

In [ ]:
%%bash

#rm -r ./playing-cards
mkdir ./playing-cards/img/train
mkdir ./playing-cards/img/validate

In [ ]:
import os
import glob
from pycocotools.coco import COCO
import random
from sklearn.model_selection import train_test_split

def create_data_file(image_list, image_type):
    with open('image-' + image_type + '.lst', 'w') as fp:
        for ind in enumerate(image_list):
            image_path = ind[1]
            
            #Move file to a new path for ease in moving to S3
            filename = image_path[20:-4]
            newpath = './playing-cards/img/' + image_type + '/' + filename + '.jpg'
        
            os.rename(image_path, newpath)
            
            fp.write(str(ind[0]) + '\t')
            if image_path.find('[W') > -1:
                fp.write('13\t')
            elif image_path.find('0]') > -1:
                fp.write('0\t')
            elif image_path.find('2]') > -1:
                fp.write('1\t')
            elif image_path.find('3]') > -1:
                fp.write('2\t')
            elif image_path.find('4]') > -1:
                fp.write('3\t')
            elif image_path.find('5]') > -1:
                fp.write('4\t')
            elif image_path.find('6]') > -1:
                fp.write('5\t')
            elif image_path.find('7]') > -1:
                fp.write('6\t')
            elif image_path.find('8]') > -1:
                fp.write('7\t')
            elif image_path.find('9]') > -1:
                fp.write('8\t')
            elif image_path.find('J]') > -1:
                fp.write('9\t')
            elif image_path.find('Q]') > -1:
                fp.write('10\t')
            elif image_path.find('K]') > -1:
                fp.write('11\t')
            elif image_path.find('A]') > -1:
                fp.write('12\t')
            
            fp.write(filename + '.jpg')
            fp.write('\n')
        fp.close()
        
        
DATA_DIR = './playing-cards/img'

SEARCH_CRITERION = '**/*.jpg'
base_images = glob.glob(os.path.join(DATA_DIR, SEARCH_CRITERION), recursive=True)

random.shuffle(base_images)

train_images, validate_images = train_test_split(base_images, test_size=0.33, random_state=0)

create_data_file(train_images, 'train')
create_data_file(validate_images, 'validate')

In [ ]:
#Re-create the train_images and validate_images files, since we moved the files
SEARCH_CRITERION = '**/*.jpg'
train_images = glob.glob(os.path.join(DATA_DIR + '/train/', SEARCH_CRITERION), recursive=True)
validate_images = glob.glob(os.path.join(DATA_DIR + '/validate/', SEARCH_CRITERION), recursive=True)


Show some sample images to make sure everything looks ok

In [ ]:
import random
from IPython.display import Image

rand_image = random.randrange(1,len(train_images))
print(train_images[rand_image])
Image(train_images[rand_image])

Push files to S3 in preparation for training

In [ ]:
# Four channels: train, validation, train_lst, and validation_lst
s3train = 's3://{}/{}/train/'.format(bucket, prefix)
s3train_lst = 's3://{}/{}/train_lst/'.format(bucket, prefix)
s3validation = 's3://{}/{}/validation/'.format(bucket, prefix)
s3validation_lst = 's3://{}/{}/validation_lst/'.format(bucket, prefix)

# upload the image files to train and validation channels
!aws s3 cp $DATA_DIR/train $s3train --recursive --quiet
!aws s3 cp $DATA_DIR/validate $s3validation --recursive --quiet
!
# upload the lst files to train_lst and validation_lst channels
!aws s3 cp image-train.lst $s3train_lst --quiet
!aws s3 cp image-validate.lst $s3validation_lst --quiet

# Training

Begin the training of our model

In [ ]:
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)
multilabel_ic = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.p3.2xlarge',
                                         train_volume_size = 50,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

In [ ]:
multilabel_ic.set_hyperparameters(num_layers=200,
                             use_pretrained_model=1,
                             image_shape = "3,224,224",
                             num_classes=14,
                             mini_batch_size=64,
                             epochs=20,
                             resize=256,
                             learning_rate=0.001,
                             num_training_samples=len(train_images),
                             use_weighted_loss=1,
                             augmentation_type = 'crop_color_transform',
                             precision_dtype='float16',
                             multi_label=0)

In [ ]:
train_data = sagemaker.session.s3_input(s3train, distribution='FullyReplicated', 
                        content_type='application/x-image', s3_data_type='S3Prefix')
train_data_lst = sagemaker.session.s3_input(s3train_lst, distribution='FullyReplicated', 
                        content_type='application/x-image', s3_data_type='S3Prefix')

validation_data = sagemaker.session.s3_input(s3validation, distribution='FullyReplicated', 
                        content_type='application/x-image', s3_data_type='S3Prefix')
validation_data_lst = sagemaker.session.s3_input(s3validation_lst, distribution='FullyReplicated', 
                        content_type='application/x-image', s3_data_type='S3Prefix')

data_channels = {'train': train_data, 'validation': validation_data, 'train_lst': train_data_lst, 
                        'validation_lst': validation_data_lst}

Start the training

In [ ]:
multilabel_ic.fit(inputs=data_channels, logs=True)

# Inference

Set up an endpoint where we can make inferences

In [ ]:
ic_classifier = multilabel_ic.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

Get a random image from our test dataset

In [ ]:
import random
from IPython.display import Image
import json

rand_image = random.randrange(1,len(validate_images)-1)
print(validate_images[rand_image])

with open(validate_images[rand_image], 'rb') as image:
    f = image.read()
    b = bytearray(f)
ic_classifier.content_type = 'application/x-image'
results = ic_classifier.predict(b)


prob = json.loads(results)
#print(prob)
classes = ['10', '2', '3', '4', '5', '6', '7', '8', '9', 'J', 'Q', 'K', 'Joker']

predicted_class = ''
predicted_prob = ''
for idx, val in enumerate(classes):
    if predicted_prob == '':
        predicted_class = classes[idx]
        predicted_prob = prob[idx]
    
    if predicted_prob < prob[idx]:
        predicted_class = classes[idx]
        predicted_prob = prob[idx]
    
    #print('%s:%f '%(classes[idx], prob[idx]), end='')

print('%s:%f '%(predicted_class, predicted_prob), end='')
Image(validate_images[rand_image])

In [ ]:
from IPython.display import Image
import json

test_image = './test/IMG_0253.jpeg'

with open(test_image, 'rb') as image:
    f = image.read()
    b = bytearray(f)
ic_classifier.content_type = 'application/x-image'
results = ic_classifier.predict(b)


prob = json.loads(results)
#print(prob)
classes = ['10', '2', '3', '4', '5', '6', '7', '8', '9', 'J', 'Q', 'K', 'Joker']

predicted_class = ''
predicted_prob = ''
for idx, val in enumerate(classes):
    if predicted_prob == '':
        predicted_class = classes[idx]
        predicted_prob = prob[idx]
    
    if predicted_prob < prob[idx]:
        predicted_class = classes[idx]
        predicted_prob = prob[idx]
    
    #print('%s:%f '%(classes[idx], prob[idx]), end='')

print('%s:%f '%(predicted_class, predicted_prob), end='')
Image(test_image)

As you can see, the model does a pretty good job at predicting the right class. This should give you an idea of how to create an image-based model. 

Clean up the endpoint

In [ ]:
ic_classifier.delete_endpoint()